In [1]:
#import nltk; nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

C:\Users\seeml\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('C:/Users/seeml/Desktop/losalamosdata.csv', error_bad_lines = False)
#print(df.target_names.unique())
df.head()

,Unnamed: 0,source_computer,destination_computer,time,source_user@domain,destination_user@domain,auth_type,longon_type,auth_orien,success/fail
0,1,C2815,C625,3721,C2815$@DOM1,C2815$@DOM1,Kerberos,Network,LogOn,Success
1,2,C14046,C754,33492,ANONYMOUS LOGON@C754,ANONYMOUS LOGON@C754,NTLM,Network,LogOn,Success
2,3,C18769,C754,816792,C18769$@DOM1,C18769$@DOM1,NTLM,Network,LogOn,Success
3,4,C5808,C18781,1770007,ANONYMOUS LOGON@C18781,ANONYMOUS LOGON@C18781,NTLM,Network,LogOn,Success
4,5,C13050,C13050,30731,C13050$@DOM1,C13050$@DOM1,?,?,TGT,Success


In [3]:
# just considering the users in source_user@domain

users_df = df[df['source_user@domain'].str.contains("U")]
users_df = users_df[-users_df['source_user@domain'].str.contains("ANONYMOUS")]
users_df = users_df[['time', 'destination_computer', 'source_user@domain']]
users_df['source_user@domain'] = users_df['source_user@domain'].str.split('@').str.get(0)

users_df

,time,destination_computer,source_user@domain
9,1254860,C2805,U2552
10,24716,C1877,U2521
27,26627,C2327,U581
32,2981235,C2721,U4995
34,3238561,C1025,U5195
42,2359124,C2058,U2398
44,35124,C743,U6230
45,26344,C2327,U2973
46,726075,C2327,U6239
51,115497,C528,U3174


In [ ]:
#U6836, U1653, U7998

In [4]:
users_df1 = users_df[users_df['source_user@domain'].str.contains("U6836|U1653|U7998")]
users_df1

,time,destination_computer,source_user@domain
128,1250211,C7521,U6836
134,1253803,C2704,U6836
165,3056451,C5666,U1653
185,1253503,C12162,U6836
222,1252208,C12129,U6836
249,1252230,C7170,U6836
274,1250965,C11246,U6836
384,1254216,C3430,U6836
441,2901918,C5666,U1653
467,3110403,C5666,U1653


In [15]:
# Create a corpus, each element represents one day of connections 

sec_day = 86400
corpus = []
for i in range(26):
    day = (users_df1["time"] <= sec_day*(i+1)) & (users_df1["time"] > sec_day*i)
    list = users_df1[day][['destination_computer', 'source_user@domain']]
    corpus.append(list)



In [17]:
# transform the dataframe into array
users_df_list=[]
for i in range(26):
    a = [x for xs in corpus[i].values for x in xs]
    users_df_list.append(a)




In [18]:
users_df_list

[['C706',
  'U1653',
  'C706',
  'U1653',
  'C2173',
  'U1653',
  'C2173',
  'U1653',
  'C231',
  'U1653',
  'C754',
  'U1653',
  'C5666',
  'U1653',
  'C231',
  'U1653',
  'C2173',
  'U1653',
  'C706',
  'U1653',
  'C231',
  'U1653',
  'C11743',
  'U7998',
  'C231',
  'U1653',
  'C706',
  'U1653',
  'C754',
  'U1653'],
 ['C2784',
  'U7998',
  'C17597',
  'U7998',
  'C17368',
  'U7998',
  'C1484',
  'U7998',
  'C12917',
  'U7998',
  'C16177',
  'U7998',
  'C17606',
  'U7998',
  'C17608',
  'U7998',
  'C12492',
  'U7998',
  'C17332',
  'U7998',
  'C404',
  'U7998',
  'C17548',
  'U7998',
  'C17579',
  'U7998',
  'C2498',
  'U7998',
  'C17448',
  'U7998',
  'C12822',
  'U7998',
  'C12704',
  'U7998',
  'C17609',
  'U7998',
  'C17566',
  'U7998',
  'C2554',
  'U7998',
  'C17538',
  'U7998',
  'C1890',
  'U7998',
  'C9477',
  'U7998',
  'C17388',
  'U7998',
  'C17500',
  'U7998',
  'C2936',
  'U7998',
  'C2266',
  'U7998',
  'C17564',
  'U7998',
  'C17528',
  'U7998',
  'C17334',
  'U7998'

26

In [20]:
#create corpus
id2word = corpora.Dictionary(users_df_list)

texts = users_df_list

corpus1 = [id2word.doc2bow(text) for text in texts]

corpus1

[[(0, 1), (1, 3), (2, 4), (3, 1), (4, 4), (5, 2), (6, 14), (7, 1)],
 [(3, 1),
  (6, 1),
  (7, 223),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),


In [21]:
##lda model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus1,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
pprint(lda_model.print_topics())

[(0,
  '0.004*"U6836" + 0.001*"C18238" + 0.001*"C18252" + 0.000*"U7998" + '
  '0.000*"U1653" + 0.000*"C231" + 0.000*"C5666" + 0.000*"C754" + 0.000*"C2173" '
  '+ 0.000*"C706"'),
 (1,
  '0.175*"U7998" + 0.027*"U1653" + 0.008*"C754" + 0.006*"C5666" + '
  '0.005*"C2173" + 0.005*"C706" + 0.004*"C231" + 0.001*"U6836" + '
  '0.001*"C17956" + 0.001*"C15475"'),
 (2,
  '0.423*"U6836" + 0.001*"U7998" + 0.001*"U1653" + 0.000*"C231" + '
  '0.000*"C23314" + 0.000*"C706" + 0.000*"C23278" + 0.000*"C23312" + '
  '0.000*"C23320" + 0.000*"C23282"')]


In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus1))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=users_df_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -4.81910779794292

Coherence Score:  0.6652264675922058


In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus1, id2word)
vis

C:\Users\seeml\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.040842 -0.157583       1        1  64.959778
5     -0.523875 -0.091455       2        1  26.441505
1     -0.053923  0.243624       3        1   4.585888
19    -0.125383  0.169254       4        1   3.367976
15     0.035326  0.034395       5        1   0.427297
16     0.042598 -0.013480       6        1   0.118417
0      0.040989 -0.011603       7        1   0.019890
18     0.043905 -0.031129       8        1   0.009788
14     0.041536 -0.010439       9        1   0.005925
12     0.041329 -0.010885      10        1   0.005876
17     0.041900 -0.017751      11        1   0.005813
7      0.041827 -0.017146      12        1   0.005813
3      0.041427 -0.011692      13        1   0.005806
9      0.041626 -0.010621      14        1   0.005795
6      0.041636 -0.010605      15        1   0.005793
13     0.041613 -0.010565      16        1   0.005788
11     0.041657 -0.010579      17        1   0.005715
8      0.041657 -0.010579      18        1   0.005715
4      0.041657 -0.010579      19        1   0.005715
2      0.041657 -0.010579      20        1   0.005715, topic_info=     Category         Freq    Term        Total  loglift  logprob
term                                                             
553   Default  6571.000000   U6836  6571.000000  30.0000  30.0000
3     Default  2550.000000   C5666  2550.000000  29.0000  29.0000
6     Default  2647.000000   U1653  2647.000000  28.0000  28.0000
7     Default   560.000000   U7998   560.000000  27.0000  27.0000
5     Default    22.000000    C754    22.000000  26.0000  26.0000
1     Default    21.000000   C2173    21.000000  25.0000  25.0000
4     Default    19.000000    C706    19.000000  24.0000  24.0000
2     Default    17.000000    C231    17.000000  23.0000  23.0000
470   Default     4.000000    C395     4.000000  22.0000  22.0000
4370  Default     2.000000  C23320     2.000000  21.0000  21.0000
4366  Default     2.000000  C23312     2.000000  20.0000  20.0000
4367  Default     2.000000  C23314     2.000000  19.0000  19.0000
4359  Default     2.000000  C23276     2.000000  18.0000  18.0000
4361  Default     2.000000  C23282     2.000000  17.0000  17.0000
4360  Default     2.000000  C23278     2.000000  16.0000  16.0000
0     Default     2.000000  C11743     2.000000  15.0000  15.0000
5414  Default     1.000000    C582     1.000000  14.0000  14.0000
1436  Default     1.000000  C12180     1.000000  13.0000  13.0000
3185  Default     1.000000  C17913     1.000000  12.0000  12.0000
5843  Default     1.000000   C6924     1.000000  11.0000  11.0000
4686  Default     1.000000     C30     1.000000  10.0000  10.0000
2792  Default     1.000000   C1638     1.000000   9.0000   9.0000
4088  Default     1.000000  C21138     1.000000   8.0000   8.0000
2931  Default     1.000000   C1682     1.000000   7.0000   7.0000
7091  Default     1.000000    C995     1.000000   6.0000   6.0000
2251  Default     1.000000  C14486     1.000000   5.0000   5.0000
5251  Default     1.000000    C498     1.000000   4.0000   4.0000
2483  Default     1.000000  C15240     1.000000   3.0000   3.0000
6304  Default     1.000000   C8045     1.000000   2.0000   2.0000
2042  Default     1.000000   C1383     1.000000   1.0000   1.0000
...       ...          ...     ...          ...      ...      ...
7334  Topic20     0.000162   C5071     1.117905   0.9315  -8.9004
2435  Topic20     0.000162   C1505     1.152644   0.9009  -8.9004
2437  Topic20     0.000162  C15057     1.154001   0.8997  -8.9004
2438  Topic20     0.000162  C15068     1.154124   0.8996  -8.9004
2439  Topic20     0.000162   C1507     1.153140   0.9005  -8.9004
2440  Topic20     0.000162  C15070     1.153046   0.9006  -8.9004
2441  Topic20     0.000162  C15075     1.153170   0.9005  -8.9004
2442  Topic20     0.000162  C15077     1.153283   0.9004  -8.9004
2443  Topic20     0.000162   C1508     1.153790   0.8999  -8.90